### Churn Modeling - NGO

In [577]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('seaborn-poster')
from collections import Counter
from scipy import stats
import seaborn as sns
import datetime as dt
from random import shuffle
from datetime import date


%matplotlib inline

In [578]:
donors = pd.read_sas('extrel.sas7bdat')

In [579]:
donors

,EXTRELNO,EXRELACTCD,EXTRELSTDT,EXRELDATEN
0,b'0026414',b'CT',1995-02-02,NaT
1,b'0026419',b'FP',1992-03-01,NaT
2,b'0026424',b'FP',1992-03-01,1996-12-24
3,b'0026430',b'FP',1992-03-01,2006-01-31
4,b'0026430',b'CT',1997-03-07,NaT
5,b'0026431',b'FP',1992-03-01,NaT
6,b'0026431',b'CT',1999-12-01,NaT
7,b'0026443',b'FP',1992-03-01,NaT
8,b'0026444',b'FP',1992-03-01,NaT
9,b'0026446',b'FP',1992-03-01,NaT


In [580]:
# renaming the columns for simplicity
donors = donors.rename(columns={'EXTRELNO' : 'Donor_Id' , 'EXRELACTCD' : 'Activity_Code',
                               'EXTRELSTDT' : 'Relation_starts' , 'EXRELDATEN' : 'Relation_ends'})

In [581]:
# lets make a new vaiable ['Left'] : 1 for relationship has ended and 0 for not.

donors['Left'] = np.where(donors['Relation_ends'].isnull() , 0 , 1)

In [582]:
donors.head()

,Donor_Id,Activity_Code,Relation_starts,Relation_ends,Left
0,b'0026414',b'CT',1995-02-02,NaT,0
1,b'0026419',b'FP',1992-03-01,NaT,0
2,b'0026424',b'FP',1992-03-01,1996-12-24,1
3,b'0026430',b'FP',1992-03-01,2006-01-31,1
4,b'0026430',b'CT',1997-03-07,NaT,0


In [583]:
len(donors['Donor_Id'].unique())

18704

In [584]:
# activity data 
activity = pd.read_sas('extrelty.sas7bdat')

In [585]:
activity = activity.rename(columns={'EXRELACTCD' : 'Activity_Code' , 'EXRELACTDE' : 'Activity name'})

In [586]:
activity

,Activity_Code,Activity name
0,b'FP',b'CommitmentDo'
1,b'PS',b'Prospect'
2,b'PF',b'Prosp Fundra'
3,b'PP',b'Prosp Promot'
4,b'GR',b'Grantor'
5,b'CT',b'Contributor'
6,b'FR',b'Friend'
7,b'AP',b'Active Press'
8,b'SP',b'Supplier'
9,b'TR',b'Translator'


In [587]:
donors = donors.merge(activity , on =['Activity_Code'] , how= 'inner')

In [588]:
donors.head()

,Donor_Id,Activity_Code,Relation_starts,Relation_ends,Left,Activity name
0,b'0026414',b'CT',1995-02-02,NaT,0,b'Contributor'
1,b'0026430',b'CT',1997-03-07,NaT,0,b'Contributor'
2,b'0026431',b'CT',1999-12-01,NaT,0,b'Contributor'
3,b'0026447',b'CT',1995-02-02,NaT,0,b'Contributor'
4,b'0026470',b'CT',1992-03-01,NaT,0,b'Contributor'


In [589]:
len(donors['Donor_Id'].unique())

18704

In [590]:
donors.isnull().sum()

Donor_Id               0
Activity_Code          0
Relation_starts        0
Relation_ends      16521
Left                   0
Activity name          0
dtype: int64

In [503]:
# importing demographic data

demographics = pd.read_sas('nameaddr.sas7bdat')

In [504]:
demographics= demographics.rename(columns={'EXTRELNO' : 'Donor_Id' ,'LANGUACODE' : 'Language' , 
                                           'NAME1TITLE' : 'Title Address' })

In [505]:
demographics

,Donor_Id,Title Address,POSTCODE,Language
0,b'0000019',b'HR',b'2500',b'NL'
1,b'0000020',b'HR',b'9550',b'NL'
2,b'0000035',b'HM',b'2150',b'NL'
3,b'0000042',b'FA',b'3012',b'NL'
4,b'0000043',b'MW',b'3000',b'NL'
5,b'0000047',b'HM',b'3360',b'NL'
6,b'0000060',b'FA',b'3600',b'NL'
7,b'0000072',b'MW',b'9000',b'NL'
8,b'0000081',b'MW',b'8902',b'NL'
9,b'0000089',b'FA',b'3600',b'NL'


In [506]:
len(demographics['Donor_Id'].unique())

24186

In [507]:
payment_history = pd.read_sas('payhistory.sas7bdat')

In [508]:
payment_history

,PID,PDATE,PAMT,EXTRELNO,PAYTYPECD,STATUS
0,38.0,1993-12-31,9.57,b'0000019',b'X',b'CO'
1,39.0,1993-12-31,41.32,b'0000020',b'X',b'CO'
2,40.0,1993-12-31,13.10,b'0000020',b'X',b'CO'
3,54091.0,1994-01-01,2.02,b'0000020',b'X',b'CO'
4,104480.0,1994-05-19,0.50,b'0000020',b'D',b'OK'
5,128703.0,1994-06-14,0.50,b'0000020',b'D',b'OK'
6,151581.0,1994-07-05,0.50,b'0000020',b'D',b'OK'
7,182688.0,1994-08-15,0.50,b'0000020',b'D',b'OK'
8,210468.0,1994-09-23,0.50,b'0000020',b'D',b'OK'
9,235968.0,1994-10-21,0.50,b'0000020',b'D',b'OK'


In [509]:
payment_history = payment_history.rename(columns = {'EXTRELNO' : 'Donor_Id' })

In [510]:
total_payment = payment_history.groupby(['Donor_Id'])['PAMT'].sum()

In [511]:
total_payment = total_payment.reset_index()

In [512]:
average_donation = payment_history.groupby(['Donor_Id'])['PAMT'].mean()
average_donation = average_donation.reset_index()

In [513]:
donation = average_donation.merge(total_payment , on =['Donor_Id'])

In [514]:
donation = donation.rename(columns = {'PAMT_x' : 'Average_Donation','PAMT_y' : 'Total Donation'})

In [515]:
donation

,Donor_Id,Average_Donation,Total Donation
0,b'0000019',9.570000,9.57
1,b'0000020',20.107405,3176.97
2,b'0000035',20.441290,3168.40
3,b'0000042',18.153294,1543.03
4,b'0000043',19.097118,4373.24
5,b'0000060',17.543816,1333.33
6,b'0000072',52.729831,3111.06
7,b'0000081',54.436901,9308.71
8,b'0000089',20.085796,3153.47
9,b'0000091',19.560342,2288.56


In [516]:
payment_history['PDATE'] = pd.to_datetime(payment_history['PDATE'])

In [517]:
payment_history['P_year'] = payment_history['PDATE'].dt.year

In [518]:
unique_id = list(payment_history['Donor_Id'].unique())

In [519]:
'''frequency_table= pd.DataFrame()
for i in unique_id:
    df = payment_history[payment_history['Donor_Id'] == i]
    a = len(df['P_year'].unique())
    new_df = df.groupby(['Donor_Id'])['P_year'].count()
    new_df = new_df.reset_index()
    new_df['Frequency'] = round(new_df['P_year'] / a )
    frequency_table = pd.concat([frequency_table , new_df] , axis = 0)
frequency_table'''
    

"frequency_table= pd.DataFrame()\nfor i in unique_id:\n    df = payment_history[payment_history['Donor_Id'] == i]\n    a = len(df['P_year'].unique())\n    new_df = df.groupby(['Donor_Id'])['P_year'].count()\n    new_df = new_df.reset_index()\n    new_df['Frequency'] = round(new_df['P_year'] / a )\n    frequency_table = pd.concat([frequency_table , new_df] , axis = 0)\nfrequency_table"

In [520]:
#frequency_table.to_csv('frequency_table.csv')

In [522]:
frequency_table

,Donor_Id,P_year,Frequency
0,b'0000019',1,1.0
0,b'0000020',158,11.0
0,b'0000035',155,10.0
0,b'0000042',85,9.0
0,b'0000043',229,21.0
0,b'0000060',76,11.0
0,b'0000072',59,4.0
0,b'0000081',171,12.0
0,b'0000089',157,10.0
0,b'0000091',117,11.0


In [523]:
donation = donation.merge(frequency_table , on = ['Donor_Id'] )

In [524]:
donation = donation.rename(columns={'P_year' : 'Total_donations'})

In [525]:
recency = payment_history.groupby(['Donor_Id'])['PDATE'].max()

In [526]:
recency = recency.reset_index()

In [527]:
donation = donation.merge(recency , on=['Donor_Id'])

In [528]:
donation = donation.rename(columns ={'PDATE' : 'Recent_donation'})

In [529]:
donation = donation.rename(columns={'Total_donations' : 'Num_donations'})

In [530]:
donation

,Donor_Id,Average_Donation,Total Donation,Num_donations,Frequency,Recent_donation
0,b'0000019',9.570000,9.57,1,1.0,1993-12-31
1,b'0000020',20.107405,3176.97,158,11.0,2007-01-17
2,b'0000035',20.441290,3168.40,155,10.0,2007-01-17
3,b'0000042',18.153294,1543.03,85,9.0,2001-01-23
4,b'0000043',19.097118,4373.24,229,21.0,2003-07-14
5,b'0000060',17.543816,1333.33,76,11.0,1999-12-07
6,b'0000072',52.729831,3111.06,59,4.0,2006-11-10
7,b'0000081',54.436901,9308.71,171,12.0,2006-12-08
8,b'0000089',20.085796,3153.47,157,10.0,2007-01-17
9,b'0000091',19.560342,2288.56,117,11.0,2003-09-15


In [531]:
a = date(2007,2,2)

In [532]:
donation['Last_donation'] = a - donation['Recent_donation']

In [533]:
# paytype
df = payment_history[['Donor_Id' , 'PAYTYPECD']]


In [534]:
df['Payment_type'] = np.where(df['PAYTYPECD'] == b'X' , 'Unknown' ,
                             np.where(df['PAYTYPECD'] == b'E' , 'Own_initiative',
                                     np.where(df['PAYTYPECD'] == b'D' , 'Permanent', 'Bank_transfer' )))

C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [535]:
def dummy_variables(df , colname):
    col_dummies = pd.get_dummies(df[colname] , prefix = colname)
    col_dummies.drop(col_dummies.columns[0] , axis = 1 , inplace = True)
    df = pd.concat([df , col_dummies] , axis =1)
    df.drop(colname, axis=1 ,inplace =True)
    return (df)
for c_feature in ['Payment_type']:
    df[c_feature] = df[c_feature].astype('category')
    df= dummy_variables(df , c_feature)

C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [536]:
df = df.groupby(['Donor_Id'])['Payment_type_Own_initiative' , 'Payment_type_Permanent' , 'Payment_type_Unknown'].mean()

In [537]:
df['Payment_Own_initiative'] = np.where(df['Payment_type_Own_initiative'] == 0 , 0 , 1)

In [538]:
df['Payment_Permanent'] = np.where(df['Payment_type_Permanent'] == 0 , 0 , 1)
df['Payment_Unknown'] = np.where(df['Payment_type_Unknown'] == 0 , 0 , 1)

In [539]:
df = df.drop(['Payment_type_Own_initiative' , 'Payment_type_Permanent' , 'Payment_type_Unknown'] , axis = 1)

In [540]:
df = df.reset_index()

In [541]:
donation = donation.merge(df , on = ['Donor_Id'])

In [542]:
donation

,Donor_Id,Average_Donation,Total Donation,Num_donations,Frequency,Recent_donation,Last_donation,Payment_Own_initiative,Payment_Permanent,Payment_Unknown
0,b'0000019',9.570000,9.57,1,1.0,1993-12-31,4781 days,0,0,1
1,b'0000020',20.107405,3176.97,158,11.0,2007-01-17,16 days,0,1,1
2,b'0000035',20.441290,3168.40,155,10.0,2007-01-17,16 days,0,1,1
3,b'0000042',18.153294,1543.03,85,9.0,2001-01-23,2201 days,0,1,1
4,b'0000043',19.097118,4373.24,229,21.0,2003-07-14,1299 days,0,1,1
5,b'0000060',17.543816,1333.33,76,11.0,1999-12-07,2614 days,0,1,1
6,b'0000072',52.729831,3111.06,59,4.0,2006-11-10,84 days,0,1,1
7,b'0000081',54.436901,9308.71,171,12.0,2006-12-08,56 days,0,1,1
8,b'0000089',20.085796,3153.47,157,10.0,2007-01-17,16 days,0,1,1
9,b'0000091',19.560342,2288.56,117,11.0,2003-09-15,1236 days,0,1,1


In [543]:
communication = pd.read_sas('communication.sas7bdat')

In [544]:
communication = communication.rename(columns = {'EXTRELNO' : 'Donor_Id'} )

In [545]:
len(communication['Donor_Id'].unique())

21932

In [546]:
communication

,CONTID,MEDIUMCODE,MNTOPCODE,CLASCODE,Donor_Id,CONTDIREC,CONTDATE
0,b'000405402',b'CI',b'CTS',b'CT',b'0000020',b'O',1994-05-09
1,b'000430904',b'CI',b'CTS',b'CT',b'0000020',b'O',1994-06-14
2,b'000592732',b'CI',b'CTS',b'CT',b'0000020',b'O',1994-10-21
3,b'000593375',b'CI',b'CTS',b'CT',b'0000020',b'O',1994-10-21
4,b'000618514',b'CI',b'CTS',b'CT',b'0000020',b'O',1994-11-28
5,b'000767603',b'LE',b'TAX',b'ID',b'0000020',b'O',1995-02-24
6,b'000890300',b'CI',b'CTS',b'CT',b'0000020',b'O',1995-05-30
7,b'000921255',b'CI',b'CTS',b'CT',b'0000020',b'O',1995-07-03
8,b'000992863',b'CI',b'CTS',b'CT',b'0000020',b'O',1995-07-10
9,b'001047202',b'CI',b'CTS',b'CT',b'0000020',b'O',1995-08-19


In [547]:
comm_medium  = pd.read_sas('commediu.sas7bdat')

In [548]:
comm_medium

,MEDIUMCODE,MEDIUMDESC
0,b'FF',b'Face to Face contact'
1,b'FX',b'Fax'
2,b'TX',b'Telex'
3,b'EM',b'Electronic Mail'
4,b'TE',b'Telephone call'
5,b'ES',b'Express mail'
6,b'AM',b'Answering machine'
7,b'RM',b'Regular mail (CTS-items only)'
8,b'AF',b'Application form'
9,b'LE',b'Letters'


In [549]:
comm_topic = pd.read_sas('commaint.sas7bdat')

In [550]:
comm_topic

,MNTOPCODE,MNTOPDESC
0,b'CAN',b'Cancellations/replacements'
1,b'DIS',b'Discontinuations'
2,b'MKT',b'Marketing - Enquiries'
3,b'GEN',b'General'
4,b'NEW',b'New relations'
5,b'PRO',b'Projects'
6,b'MFC',"b'Mail met FC, klachten en info'"
7,b'SPC',b'Special Forms'
8,b'INF',b'information about Plan'
9,b'ENR',b'Enrollment'


In [551]:
comm_class = pd.read_sas('comclas.sas7bdat')

In [552]:
comm_class

,CLASCODE,CLASDESC
0,b'CT',b'CTS-item'
1,b'RI',b'Request for information'
2,b'CM',b'Complaint'
3,b'TA',b'Threath / Accusation'
4,b'AK',b'Acknowledgement'
5,b'RM',b'Reminder'
6,b'ID',b'Information delivery'
7,b'AN',b'Answer'


In [553]:
complaint_donors = communication[['Donor_Id' ,'CLASCODE']]

In [554]:
complaint_donors['Complaint_reg'] = np.where(complaint_donors['CLASCODE'] == b'CM' , 1 , 0) 

C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [555]:
complaint_donors = complaint_donors.groupby(['Donor_Id'])['Complaint_reg'].mean()

In [556]:
complaint_donors = complaint_donors.reset_index()

In [557]:
complaint_donors['Complaint_done'] = np.where(complaint_donors['Complaint_reg'] == 0 , 0 , 1)

In [558]:
complaint_donors = complaint_donors.drop(['Complaint_reg'] , axis = 1)

In [559]:
complaint_donors.head()

,Donor_Id,Complaint_done
0,b'0000020',0
1,b'0000035',0
2,b'0000042',0
3,b'0000043',0
4,b'0000060',0


In [560]:
incoming = communication[['Donor_Id' , 'CONTDIREC']]

In [561]:
incoming['Incoming'] = np.where(incoming['CONTDIREC'] == b'O' , 0 , 1)

C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [562]:
incoming = incoming.groupby(['Donor_Id'])['Incoming'].mean()

In [563]:
incoming = incoming.reset_index()

In [564]:
incoming['Incoming_comm'] = np.where(incoming['Incoming'] == 0 , 0 ,1)

In [565]:
incoming = incoming.drop(['Incoming'] , axis = 1)

In [566]:
incoming.head()

,Donor_Id,Incoming_comm
0,b'0000020',0
1,b'0000035',1
2,b'0000042',1
3,b'0000043',1
4,b'0000060',1


In [603]:
info_donors = donors[['Donor_Id'  , 'Left' , 'Activity_Code']]

In [604]:
info_donors = info_donors.groupby(['Donor_Id' , 'Activity_Code'])['Left'].sum()

In [605]:
info_donors = info_donors.reset_index()

In [606]:
info_donors

,Donor_Id,Activity_Code,Left
0,b'0026414',b'CT',0
1,b'0026419',b'FP',0
2,b'0026424',b'FP',1
3,b'0026430',b'CT',0
4,b'0026430',b'FP',1
5,b'0026431',b'CT',0
6,b'0026431',b'FP',0
7,b'0026443',b'FP',0
8,b'0026444',b'FP',0
9,b'0026446',b'FP',0


In [434]:
total_donors = demographics[['Donor_Id' , 'Language']]

In [435]:
total_donors = total_donors.merge(info_donors , on = ['Donor_Id'] , how = 'outer')

In [437]:
total_donors.head()

,Donor_Id,Language,Left
0,b'0000019',b'NL',NaN
1,b'0000020',b'NL',NaN
2,b'0000035',b'NL',NaN
3,b'0000042',b'NL',NaN
4,b'0000043',b'NL',NaN


In [440]:
total_donors = total_donors.merge(donation , on = ['Donor_Id'] , how = 'outer')

In [442]:
total_donors = total_donors.drop(['Recent_donation'] , axis = 1)

In [476]:
total_donors = total_donors.merge(complaint_donors , on = ['Donor_Id'] , how = 'left')

In [478]:
total_donors = total_donors.merge(incoming , on = ['Donor_Id'] , how = 'left')

In [481]:
total_donors.head()

,Donor_Id,Language,Left,Average_Donation,Total Donation,Num_donations,Frequency,Last_donation,Payment_Own_initiative,Payment_Permanent,Payment_Unknown,Complaint_done,Incoming_comm
0,b'0000019',b'NL',NaN,9.570000,9.57,1.0,1.0,4781 days,0.0,0.0,1.0,NaN,NaN
1,b'0000020',b'NL',NaN,20.107405,3176.97,158.0,11.0,16 days,0.0,1.0,1.0,0.0,0.0
2,b'0000035',b'NL',NaN,20.441290,3168.40,155.0,10.0,16 days,0.0,1.0,1.0,0.0,1.0
3,b'0000042',b'NL',NaN,18.153294,1543.03,85.0,9.0,2201 days,0.0,1.0,1.0,0.0,1.0
4,b'0000043',b'NL',NaN,19.097118,4373.24,229.0,21.0,1299 days,0.0,1.0,1.0,0.0,1.0


In [483]:
from matplotlib.backends.backend_pdf import PdfPages
import pandas_profiling

C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\pandas_profiling\plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\SHIVAM-PC\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\SHIVAM-PC\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\SHIVAM-PC\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.ins

In [484]:
pandas_profiling.ProfileReport(total_donors)

Number of variables,13
Number of observations,33342
Total Missing (%),17.3%
Total size in memory,3.6 MiB
Average record size in memory,112.0 B
Numeric,10
Categorical,3
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [485]:
len(total_donors['Donor_Id'].unique())

24186